# Required Assignment 17.1 — Comparing Classifiers (Bank Marketing)
Nicholas Cross

**Goal:** Compare k-nearest neighbors (KNN), logistic regression, decision trees, and support vector machines (SVM) to predict whether a client subscribes to a term deposit (`y`).

## 1. Business Problem
The bank runs outbound phone campaigns to offer term deposits. Calls have a cost (agent time) and a potential payoff (subscription). We want to **prioritize which clients to call** to maximize conversions and reduce wasted calls.

This is a **binary classification** problem. Because the positive class (`y=yes`) is typically rarer, we focus on **F1** and **recall** for the positive class, not accuracy alone.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from scipy.stats import chi2_contingency, ttest_ind

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    f1_score, recall_score, precision_score, roc_auc_score, RocCurveDisplay
)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


## 2. Load Data
We use the UCI Bank Marketing dataset (`bank-additional-full.csv`).

In [ ]:
# Load dataset (robust path handling)
candidate_paths = [
    Path("data/bank-additional-full.csv"),
    Path("./data/bank-additional-full.csv"),
    Path("../data/bank-additional-full.csv"),
]

data_path = None
for p in candidate_paths:
    if p.exists():
        data_path = p
        break

if data_path is None:
    raise FileNotFoundError("Could not find bank-additional-full.csv")

df = pd.read_csv(data_path, sep=';')
df.shape, df.head()

## 3. Data Understanding
### 3.1 Target distribution (class imbalance)

In [ ]:
target_counts = df["y"].value_counts()
target_pct = df["y"].value_counts(normalize=True) * 100
display(pd.DataFrame({"count": target_counts, "percent": target_pct.round(2)}))

plt.figure(figsize=(6,4))
plt.bar(target_counts.index.astype(str), target_counts.values)
plt.title("Target Distribution (y)")
plt.xlabel("Subscribed to term deposit?")
plt.ylabel("Count")
plt.show()

### 3.2 Descriptive statistics

In [ ]:
numeric_cols = df.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
numeric_cols, cat_cols[:5], len(cat_cols)

In [ ]:
df[numeric_cols].describe().T

### 3.3 Key plots
**Note on leakage:** `duration` (call length) is known after/during the call. It may be unusable for deciding *who to call*.
We will evaluate models both **with** and **without** `duration`.

In [ ]:
# Age distribution by outcome
plt.figure(figsize=(7,4))
for label in ["no", "yes"]:
    subset = df.loc[df["y"] == label, "age"]
    plt.hist(subset, bins=30, alpha=0.5, density=True, label=label)
plt.title("Age Distribution by Outcome")
plt.xlabel("Age")
plt.ylabel("Density")
plt.legend(title="y")
plt.show()

# Duration boxplot by outcome
plt.figure(figsize=(7,4))
df.boxplot(column="duration", by="y")
plt.title("Call Duration by Outcome")
plt.suptitle("")
plt.xlabel("Subscribed?")
plt.ylabel("Duration (seconds)")
plt.show()

# Previous campaign outcome vs subscription
ct = pd.crosstab(df["poutcome"], df["y"])
ct.plot(kind="bar", figsize=(10,4))
plt.title("Previous Campaign Outcome vs Subscription")
plt.xlabel("poutcome")
plt.ylabel("Count")
plt.legend(title="y")
plt.tight_layout()
plt.show()

## 4. Inferential statistics (quick checks)
- **Chi-square** tests for association between selected categorical variables and `y`
- A simple mean comparison (Welch t-test) for `age` across outcomes

In [ ]:
cat_to_test = ["job", "marital", "education", "contact", "month", "poutcome"]

chi_results = []
for c in cat_to_test:
    ct = pd.crosstab(df[c], df["y"])
    chi2, p, dof, exp = chi2_contingency(ct)
    chi_results.append({"feature": c, "chi2": chi2, "dof": dof, "p_value": p})

pd.DataFrame(chi_results).sort_values("p_value")

In [ ]:
age_yes = df.loc[df["y"]=="yes", "age"]
age_no = df.loc[df["y"]=="no", "age"]
t_stat, p_val = ttest_ind(age_yes, age_no, equal_var=False)

summary = pd.DataFrame({
    "group": ["yes", "no"],
    "mean_age": [age_yes.mean(), age_no.mean()],
    "std_age": [age_yes.std(), age_no.std()],
    "n": [len(age_yes), len(age_no)]
})
summary, (t_stat, p_val)

## 5. Preprocessing
We encode the target as 1/0, one-hot encode categoricals, and scale numerics. We keep a stratified train/test split.

We build:
1) `X_all` (includes `duration`)
2) `X_no_duration` (excludes `duration` for pre-call targeting)

In [ ]:
df_model = df.copy()
df_model["y"] = (df_model["y"] == "yes").astype(int)

X_all = df_model.drop(columns=["y"])
y = df_model["y"]
X_no_duration = X_all.drop(columns=["duration"])

X_train_all, X_test_all, y_train, y_test = train_test_split(
    X_all, y, test_size=0.2, random_state=42, stratify=y
)
X_train_nodur, X_test_nodur, _, _ = train_test_split(
    X_no_duration, y, test_size=0.2, random_state=42, stratify=y
)

numeric_cols_all = X_all.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols_all = X_all.select_dtypes(include=["object"]).columns.tolist()

numeric_cols_nodur = X_no_duration.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols_nodur = X_no_duration.select_dtypes(include=["object"]).columns.tolist()

X_train_all.shape, X_train_nodur.shape

## 6. Modeling + Evaluation
We use **Stratified 5-fold CV** and **GridSearchCV**.
Primary selection metric: **F1** for the positive class (`y=1`).

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def build_preprocessor(numeric_cols, cat_cols):
    return ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ],
        remainder="drop"
    )

def evaluate_best_model(best_estimator, X_test, y_test, title="Model"):
    y_pred = best_estimator.predict(X_test)

    y_proba = None
    if hasattr(best_estimator, "predict_proba"):
        y_proba = best_estimator.predict_proba(X_test)[:, 1]
    elif hasattr(best_estimator, "decision_function"):
        scores = best_estimator.decision_function(X_test)
        y_proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)

    print(f"=== {title} ===")
    print(classification_report(y_test, y_pred, digits=3))

    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm).plot(values_format="d")
    plt.title(f"Confusion Matrix — {title}")
    plt.show()

    if y_proba is not None:
        RocCurveDisplay.from_predictions(y_test, y_proba)
        plt.title(f"ROC Curve — {title}")
        plt.show()

    return {
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
    }

## 7. Train + Tune Models (with `duration`)
Upper-bound performance.

In [ ]:
pre_all = build_preprocessor(numeric_cols_all, cat_cols_all)

models = {
    "KNN": (KNeighborsClassifier(), {
        "model__n_neighbors": [5, 15],
        "model__weights": ["uniform", "distance"],
    }),
    "LogisticRegression": (LogisticRegression(max_iter=2000, class_weight="balanced"), {
        "model__C": [0.5, 1.0, 2.0],
    }),
    "DecisionTree": (DecisionTreeClassifier(class_weight="balanced", random_state=42), {
        "model__max_depth": [3, 5, None],
        "model__min_samples_split": [2, 20],
        "model__min_samples_leaf": [1, 10],
    }),
    "SVC_RBF": (SVC(kernel="rbf", probability=True, class_weight="balanced"), {
        "model__C": [1.0, 5.0],
        "model__gamma": ["scale", 0.1],
    })
}

results_all = []
best_models_all = {}

for name, (est, param_grid) in models.items():
    pipe = Pipeline(steps=[("pre", pre_all), ("model", est)])
    gs = GridSearchCV(pipe, param_grid=param_grid, cv=cv, scoring="f1", n_jobs=-1, verbose=0)
    gs.fit(X_train_all, y_train)

    best_models_all[name] = gs.best_estimator_
    results_all.append({"model": name, "best_f1_cv": gs.best_score_, "best_params": gs.best_params_})

pd.DataFrame(results_all).sort_values("best_f1_cv", ascending=False)

In [ ]:
test_metrics_all = []
for name, best_est in best_models_all.items():
    metrics = evaluate_best_model(best_est, X_test_all, y_test, title=f"{name} (with duration)")
    metrics["model"] = name
    test_metrics_all.append(metrics)

pd.DataFrame(test_metrics_all).set_index("model").sort_values("f1", ascending=False)

## 8. Train + Tune Models (without `duration`)
More realistic for pre-call targeting.

In [ ]:
pre_nodur = build_preprocessor(numeric_cols_nodur, cat_cols_nodur)

results_nodur = []
best_models_nodur = {}

for name, (est, param_grid) in models.items():
    pipe = Pipeline(steps=[("pre", pre_nodur), ("model", est)])
    gs = GridSearchCV(pipe, param_grid=param_grid, cv=cv, scoring="f1", n_jobs=-1, verbose=0)
    gs.fit(X_train_nodur, y_train)

    best_models_nodur[name] = gs.best_estimator_
    results_nodur.append({"model": name, "best_f1_cv": gs.best_score_, "best_params": gs.best_params_})

pd.DataFrame(results_nodur).sort_values("best_f1_cv", ascending=False)

In [ ]:
test_metrics_nodur = []
for name, best_est in best_models_nodur.items():
    metrics = evaluate_best_model(best_est, X_test_nodur, y_test, title=f"{name} (no duration)")
    metrics["model"] = name
    test_metrics_nodur.append(metrics)

pd.DataFrame(test_metrics_nodur).set_index("model").sort_values("f1", ascending=False)

## 9. Interpretability
### 9.1 Logistic Regression coefficients (no-duration)
We show the largest positive/negative coefficients as a stakeholder-friendly explanation.

In [ ]:
def get_feature_names(preprocessor, numeric_cols, cat_cols):
    ohe = preprocessor.named_transformers_["cat"]
    cat_feature_names = ohe.get_feature_names_out(cat_cols)
    return np.concatenate([np.array(numeric_cols), cat_feature_names])

lr_best = best_models_nodur["LogisticRegression"]
pre = lr_best.named_steps["pre"]
model = lr_best.named_steps["model"]

feature_names = get_feature_names(pre, numeric_cols_nodur, cat_cols_nodur)
coef_df = pd.DataFrame({"feature": feature_names, "coef": model.coef_.ravel()})

top_pos = coef_df.sort_values("coef", ascending=False).head(15)
top_neg = coef_df.sort_values("coef", ascending=True).head(15)

top_pos, top_neg

### 9.2 Decision Tree feature importances (no-duration)

In [ ]:
dt_best = best_models_nodur["DecisionTree"]
pre = dt_best.named_steps["pre"]
tree = dt_best.named_steps["model"]

feature_names = get_feature_names(pre, numeric_cols_nodur, cat_cols_nodur)
imp_df = pd.DataFrame({"feature": feature_names, "importance": tree.feature_importances_}).sort_values("importance", ascending=False)
imp_df.head(20)

## 10. Findings (Actionable Insights)
- **Don’t use accuracy alone** due to class imbalance; use **F1/recall**.
- `duration` boosts performance but is not usable for *pre-call* decisions.
- For *pre-call targeting*, pick the best **no-duration** model to rank customers for calling.
- Keep an interpretable model (Logistic Regression or shallow Decision Tree) for explaining drivers.

**Recommendation:** Use model scores to build a prioritized call list, then tune the decision threshold based on how many calls the team can place per day.

## 11. Next Steps
- Add cost-sensitive evaluation (profit per call) using business values for TP/FP/FN.
- Calibrate probabilities and tune thresholds to match campaign capacity.
- Benchmark ensembles (Random Forest / Gradient Boosting) as a follow-up.
- Consider a temporal split if campaigns are time-dependent (avoid look-ahead bias).